In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary
import pandas as pd
import numpy as np

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout_val = 0.1
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0) #input 28, Output 26, RF: 3 
        self.bn1   = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=0) #input 26, Output 24, RF: 5
        self.bn2   = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=0) #input 24, Output 22, RF: 7
        self.bn3   = nn.BatchNorm2d(32)

        self.dropout1 = nn.Dropout2d(self.dropout_val)
        
        self.pool1 = nn.MaxPool2d(2, 2) #input 22, Output 11, RF: 9

        self.conv4 = nn.Conv2d(32, 16, 3, padding=0) #input 11, Output 9, RF: 13
        self.bn4   = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=0) #input 9, Output 7, RF: 17
        self.bn5   = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3) #input 7, Output 5, RF: 21
        self.bn6   = nn.BatchNorm2d(16)
        
        self.dropout2 = nn.Dropout2d(self.dropout_val)

        self.conv7 = nn.Conv2d(16, 10, 3) #input 5, Output 3, RF: 25 
        self.bn7   = nn.BatchNorm2d(10)
        self.conv8 = nn.Conv2d(10, 10, 3) #input 3, Output 1, RF: 29

        self.gap_linear = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Conv2d(16, 10, 1)
        )

    def forward(self, x):
        x = F.relu(self.dropout1(self.bn3(self.conv3(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x))))))))))
        x = self.pool1(x)
        x = F.relu(self.dropout2(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(F.relu(self.bn4(self.conv4(x))))))))))
        x = self.gap_linear(x)
        #x = F.relu(self.bn7(self.conv7(x)))
        #x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [7]:
!pip install torchsummary


In [8]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,320
       BatchNorm2d-4           [-1, 16, 24, 24]              32
            Conv2d-5           [-1, 32, 22, 22]           4,640
       BatchNorm2d-6           [-1, 32, 22, 22]              64
         Dropout2d-7           [-1, 32, 22, 22]               0
         MaxPool2d-8           [-1, 32, 11, 11]               0
            Conv2d-9             [-1, 16, 9, 9]           4,624
      BatchNorm2d-10             [-1, 16, 9, 9]              32
           Conv2d-11             [-1, 16, 7, 7]           2,320
      BatchNorm2d-12             [-1, 16, 7, 7]              32
           Conv2d-13             [-1, 16, 5, 5]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    pbar = tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    print('\nTraining Set: Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest Set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1, 20):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.




Training Set: Accuracy: 36775/60000 (61.29%)


Test Set: Average loss: 0.6270, Accuracy: 9140/10000 (91.40%)

Epoch:  2




Training Set: Accuracy: 53773/60000 (89.62%)


Test Set: Average loss: 0.1942, Accuracy: 9704/10000 (97.04%)

Epoch:  3




Training Set: Accuracy: 55957/60000 (93.26%)


Test Set: Average loss: 0.1227, Accuracy: 9754/10000 (97.54%)

Epoch:  4


...



Training Set: Accuracy: 58611/60000 (97.69%)


Test Set: Average loss: 0.0334, Accuracy: 9899/10000 (98.99%)

Epoch:  16




Training Set: Accuracy: 58619/60000 (97.70%)


Test Set: Average loss: 0.0313, Accuracy: 9908/10000 (99.08%)

Epoch:  17




Training Set: Accuracy: 58638/60000 (97.73%)


Test Set: Average loss: 0.0314, Accuracy: 9906/10000 (99.06%)

Epoch:  18




Training Set: Accuracy: 58705/60000 (97.84%)


Test Set: Average loss: 0.0325, Accuracy: 9906/10000 (99.06%)

Epoch:  19




Training Set: Accuracy: 58716/60000 (97.86%)


Test Set: Average loss: 0.0279, Accuracy: 9917/10000 (99.17%)

